# Map CSV together and remove elsewhere

This file will combine the participants csv together with the fixations and ROI csv.


In [1]:
import pandas as pd
import pandas as pd
import seaborn as sns
import cv2
from matplotlib import pyplot as plt
from random import choice
import numpy as np
from collections import Counter

The fixations that are outside the picture get removed since they do not contribute to the overal research.


In [ ]:
pad = "../../data/csv/"
image_pad =  "../../data/Images/ROIs/"

participants = pd.read_csv(pad+"participants.csv")
fixations = pd.read_csv(pad+"fixations.csv")
ROI = pd.read_csv(pad + "ROIs.csv")

fix = fixations.merge(participants, how="left", on="Participant")
#remove fixations outside the image
fix = fix[(fix["X"] > 0) & (fix["X"] < 1024) & (fix["Y"] > 0) & (fix["Y"] < 768)]

filtered_fix = fix.copy()

After removing the fixations that are out of the image, there are still 2550 fixations that are elsewhere in the data.


In [3]:
len(filtered_fix[(filtered_fix["Name"] == "Elsewhere")])

2641

To reduce this amount, a function is created. This function recieves the old x and y, the image and scene for which we want a new ROI and the pixel offset. We take this x and y and offset is by the offset. We then look at the rgb value at this location and look in our ROI csv to pick the corresponding ROI. If we don't find it, we retry with a bigger offset. Finally the new ROI is returned.


In [4]:
def get_circular_coordinates(X, Y, offset=20):
    # List to store pixels within the radius
    selected_pixels = []
    
    # Loop through a bounding box around the center pixel
    for x in range(X - offset, X + offset + 1):
        for y in range(Y - offset, Y + offset + 1):
            # Compute Euclidean distance
            if np.sqrt((x - X) ** 2 + (y - Y) ** 2) <= offset:
                selected_pixels.append((x, y))

# Print or use the selected pixels
    return selected_pixels

In [5]:
def get_name_from_coordinates(X, Y, image, scene, offset=1):
    img = cv2.imread(image_pad + f'scene{scene}_image{image}.png')
    coordinates = get_circular_coordinates(int(X), int(Y))
    rois = []
    for coordinate in coordinates:
        if (0 <= coordinate[1] < img.shape[0]) and (0 <= coordinate[0] < img.shape[1]):
            B, G, R = img[coordinate[1], coordinate[0], :]
            this_ROI = ROI[(ROI["C1"] == R) & (ROI["C2"] == G) & (ROI["C3"] == B) & (ROI["Image"] == image) &  (ROI["Scene"] == scene)]
            if len(this_ROI) > 0:
                rois.append(this_ROI.iloc[0]["Name"])
    # If no valid ROIs found, return None
    if not rois:
        print("No ROIs found!")
        return None
    # Count occurrences of each ROI
    roi_counts = Counter(rois)
    # Ensure roi_counts is not empty before accessing elements
    if not roi_counts:
        print("No common ROI found!")
        return 'Elsewhere'
    # Find the most common ROI
    most_common_roi, count = roi_counts.most_common(1)[0]
    return most_common_roi

The function above is only helpfull if we actually use it so that's what the code below does. The dataset gets filtered on elsewhere, the necessary columns are given to the function and the result get's saved in the Name column.


In [6]:
filtered_fix['Name'] = filtered_fix.apply(lambda row: get_name_from_coordinates(row['X'], row['Y'], row['Image'], row['Scene']) , axis=1)

No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs fo

Here the same happens as above but we specifically look at fixations that have NaN in the name column.


In [7]:
filtered_fix['Name'] = filtered_fix.apply(lambda row: get_name_from_coordinates(row['X'], row['Y'], row['Image'], row['Scene']) 
                      if pd.isna(row['Name']) else row['Name'], axis=1)

No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs found!
No ROIs fo

The amount of fixations that are elsewhere have now succesfully been reduced from 2550 to 75.


In [8]:
len(filtered_fix[(filtered_fix["Name"] == "Elsewhere")]) 

0

As a final thing, the remaining fixations with elsewhere gets removed.


In [9]:
fix_filtered = filtered_fix[filtered_fix['Name'] != 'Elsewhere']

Lastly we save all of this to a new csv.


In [10]:
fix_filtered.to_csv(pad + "cleaned_fix_test_all.csv")

Because the evidence, exitentry and movable columns are not in the fixations csv we add these from the ROI csv.


In [11]:
fix_evidence = fix_filtered.merge(ROI[['Name', 'Evidence', 'Movable', 'ExitEntry', 'Image', 'Scene']], how='left', on=['Name', 'Image', 'Scene'])

fix_evidence.head()

,Image,Scene,ROI,Participant,FixDur,FixNr,Start,End,X,Y,Name,NextROI,experience,Evidence,Movable,ExitEntry
0,1,1,0,2.0,103.0,1.0,7.0,109.0,510.9,381.0,Back_wall,Bottle_on_table,Control,No,No,No
1,1,1,22,2.0,77.0,2.0,236.0,312.0,450.8,307.0,Curtains,Floor,Control,No,No,No
2,1,1,9,2.0,300.0,3.0,347.0,646.0,414.5,447.9,Back_chair,Body,Control,No,No,No
3,1,1,8,2.0,165.0,4.0,687.0,851.0,468.6,619.1,Floor,Elsewhere,Control,No,No,No
4,1,1,0,2.0,461.0,5.0,876.0,1336.0,465.4,687.9,Body,Elsewhere,Control,Yes,Yes,No


In [12]:
fix_evidence.to_csv(pad + "fix_evidence_test_all.csv")